In [21]:
# ------------------ 1. 🔐 Environment Setup ------------------
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from neo4j import GraphDatabase
import pandas as pd

# Load environment variables
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [22]:
# ------------------ 2. 🔗 Initialize the language model & Neo4j & Query Chain------------------
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=OPENAI_API_KEY
)

# Connect to Neo4j
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Build the query chain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=False,
    allow_dangerous_requests=True
)

In [29]:
# ------------------ 3. 🔗 Streamlit App ------------------
# App layout
st.set_page_config(page_title="Netflix Q&A", layout="centered")
st.title("🎬 Data_Agent: Ask Netflix Dataset")
st.markdown("Ask anything about the Netflix movies dataset.")

# Session state defaults
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

if "session_name" not in st.session_state:
    st.session_state.session_name = ""

# Sidebar session input
st.sidebar.header("🗂️ Session Info")
session_input = st.sidebar.text_input("Enter Session Name:", placeholder="e.g. Demo with Client")

if session_input:
    st.session_state.session_name = session_input
    st.sidebar.success(f"Current Session: {session_input}")

2025-04-25 00:15:18.497 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 00:15:18.501 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 00:15:18.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 00:15:18.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 00:15:18.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 00:15:18.505 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 00:15:18.506 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 00:15:18.507 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
# Query input
user_input = st.text_input("Enter your query:", placeholder="e.g. Find the most recent Indian movie")

if user_input:
    with st.spinner("Thinking..."):
        try:
            # Get LLM response
            response = chain.invoke({"query": user_input})
            st.success("Answer:")
            st.write(response)

            # Save in memory
            st.session_state.chat_history.append({
                "query": user_input,
                "answer": response
            })

            # Optional: Show Cypher + snapshot
            if st.button("🔍 Show Cypher + Data Snapshot"):
                cypher = chain.cypher_generation_chain.run({"question": user_input})
                st.code(cypher, language="cypher")

                try:
                    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
                    with driver.session() as session:
                        records = session.run(cypher)
                        df = pd.DataFrame([r.data() for r in records])

                        if not df.empty:
                            st.info("📊 Matching Data Snapshot:")
                            st.dataframe(df)
                        else:
                            st.warning("No matching data found for this query.")
                except Exception as db_err:
                    st.error(f"Error while fetching data snapshot: {db_err}")

        except Exception as e:
            st.error(f"❌ Error: {e}")

In [26]:
# Optional: Show full memory for this session
if st.session_state.chat_history:
    with st.expander("🧠 View Chat Memory"):
        for i, chat in enumerate(st.session_state.chat_history):
            st.markdown(f"**Q{i+1}:** {chat['query']}")
            st.markdown(f"**A{i+1}:** {chat['answer']}")
            st.markdown("---")

2025-04-24 23:23:41.134 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
